# Deploying Dask clusters

## 1) local machines

### 1.1) Threads on local machine

In [2]:
import dask.dataframe as dd

In [3]:
# Compute stuff directly
# df = dd.read_csv(...)
# df.x.sum().compute()  # This uses threads on your local machine

In [ ]:
# Explicitely configure Dask to use multi-threading
dask.config.set(scheduler='threads')

### 1.2) Multiprocessing on local machine

The multi-process backend is able to avoid Python’s global interpreter lock by launching separate processes. Launching a new process is **more expensive than a new thread**, and Dask needs to serialize data that moves between processes.3

In [4]:
from dask.distributed import LocalCluster
cluster = LocalCluster()          # Fully-featured local Dask cluster
client = cluster.get_client()

# Dask works as normal and leverages the infrastructure defined above
# df.x.sum().compute()

/home/nsoler/.cache/pypoetry/virtualenvs/daskscratch-1ORGWtuD-py3.12/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37643 instead
  warnings.warn(


In [ ]:
# additionally, you can configure dask to use multiprocessing by default on your local backend
dask.config.set(scheduler='processes')

# Using the forkserver will not reduce the communication overhead.
dask.config.set({"multiprocessing.context": "forkserver",
                "scheduler": "processes"})

The LocalCluster cluster manager defined above is easy to use and works well on a single machine. It follows the **same interface** as all other Dask cluster managers, and so it’s easy to swap out when you’re ready to scale up.

## 2) Kubernetes

In [5]:
from dask_kubernetes import KubeCluster

# cluster = LocalCluster()
cluster = KubeCluster()  # example, you can swap out for Kubernetes

client = cluster.get_client()

ModuleNotFoundError: No module named 'dask_kubernetes'

## 3) Cloud

Deploying on commercial cloud like AWS, GCP, or Azure is convenient because you can quickly scale out to many machines for just a few minutes, but also challenging because you need to navigate awkward cloud APIs, manage remote software environments with Docker, send data access credentials, make sure that costly resources are cleaned up, etc. The following solutions help with this process.

In [7]:
# Coiled (recommended): this commercial SaaS product handles most of the deployment pain Dask users encounter, 
# is easy to use, and quite robust. The free tier is large enough for most individual users, even for those who don’t want to engage with a commercial company. The API looks like the following.

import coiled
cluster = coiled.Cluster(
    n_workers=100,
    region="us-east-2",
    worker_memory="16 GiB",
    spot_policy="spot_with_fallback",
)
client = cluster.get_client()

# Dask Cloud Provider: a pure and simple OSS solution that sets up Dask workers on cloud VMs, supporting AWS, GCP, Azure, 
# and also other commercial clouds like Hetzner, Digital Ocean and Nebius.

# Dask-Yarn: deploys Dask on legacy YARN clusters, such as can be set up with AWS EMR or Google Cloud Dataproc.

ModuleNotFoundError: No module named 'coiled'

## 4) HPC

In [ ]:
# Dask runs on traditional HPC systems that use a resource manager like SLURM, PBS, SGE, LSF, or similar systems,
# and a network file system. This is an easy way to dual-purpose large-scale hardware for analytics use cases. 
# Dask can deploy either directly through the resource manager or through mpirun/mpiexec and 
# tends to use the NFS to distribute data and software.

# Dask-Jobqueue (recommended): interfaces directly with the resource manager (SLURM, PBS, SGE, LSF, and others)
# to launch many Dask workers as batch jobs. It generates batch job scripts and submits them automatically to the user’s queue.
# This approach operates entirely with user permissions (no IT support required) and enables interactive and adaptive use on large HPC systems.
# It looks a little like the following:

from dask_jobqueue import PBSCluster

cluster = PBSCluster(cores=36,
                     memory="100GB",
                     project='P48500028',
                     queue='premium',
                     interface='ib0',
                     walltime='02:00:00')

cluster.scale(100)  # Start 100 workers in 100 jobs that match the description above

from dask.distributed import Client
client = Client(cluster)    # Connect to that cluster

# Dask-MPI: deploys Dask on top of any system that supports MPI using mpirun. It is helpful for batch processing jobs where you want to ensure a fixed and stable number of workers.

# Dask Gateway for Jobqueue: Multi-tenant, secure clusters. Once configured, users can launch clusters without direct access to the underlying HPC backend.

More details for HPC on [this page](https://docs.dask.org/en/stable/deploying-hpc.html)

In [ ]:
# You can also use dask_mpi (better suited for batch jobs)

# set up a scheduler
mpirun --np 4 dask-mpi --scheduler-file /home/$USER/scheduler.json

# start the cluster, providing the scheduler address
from dask.distributed import Client
client = Client(scheduler_file='/path/to/scheduler.json')

# This depends on the mpi4py library. It only uses MPI to start the Dask cluster and not for inter-node communication. 
# MPI implementations differ: the use of mpirun --np 4 is specific to the mpich or open-mpi MPI implementation installed through 
# conda and linked to mpi4py.

# conda install mpi4py

## 5) Dask gateway

In [ ]:
# Dask Gateway provides a secure, multi-tenant server for managing Dask clusters.
# It allows users to launch and use Dask clusters in a shared, centrally managed cluster environment,
# without requiring users to have direct access to the underlying cluster backend (e.g. Kubernetes, Hadoop/YARN, HPC Job queues, etc…).

# helm install --repo https://helm.dask.org --create-namespace -n dask-gateway --generate-name dask-gateway

from dask_gateway import Gateway
gateway = Gateway("<gateway service address>")
cluster = gateway.new_cluster()

# This is a good choice if you want to do the following:
#  - Abstract users away from Kubernetes.

# - Provide a consistent Dask user experience across Kubernetes/Hadoop/HPC.

# Learn more at gateway.dask.org.

## 6) Command line

See https://docs.dask.org/en/stable/deploying-cli.html

## 7) Autoscaling


Auto-scaling

With auto-scaling, Dask can increase or decrease the computers/resources being used, based on the tasks you have asked it to run. For example, if you have a program that computes complex aggregations using many computers but then mostly operates on the aggregated data, the number of computers you need could decrease by a large amount post-aggregation. Many workloads, including machine learning, do not need the same amount of resources/computers the entire time.

Some of Dask’s cluster backends, including Kubernetes, support auto-scaling, which Dask calls adaptive deployments. Auto-scaling is useful mostly in situations of shared cluster resources, or when running on cloud providers where the underlying resources are paid for by the hour.


## 8) data serialization

Part of why Dask is so powerful is the Python ecosystem that it is in. While Dask will pickle, or serialize (see “Serialization and Pickling”), and send our code to the workers, this doesn’t include the libraries we use.6 To take advantage of that ecosystem, you need to be able to use additional libraries. During the exploration phase, it is common to install packages at runtime as you discover that you need them.

The PipInstall worker plug-in takes a list of packages and installs them at runtime on all of the workers. Looking back at Example 2-4, to install bs4 you would call distributed.diagnostics.plugin.PipInstall(["bs4"]). Any new workers that are launched by Dask then need to wait for the package to be installed. The Pip​Install plug-in is ideal for quick prototyping when you are discovering which packages you need. You can think of PipInstall as the replacement for !pip install in a notebook over having a virtualenv.

To avoid the slowness of having to install packages each time a new worker is launched, you should try to pre-install your libraries. Each cluster manager (e.g., YARN, Kubernetes, Coiled, Saturn, etc.) has its own methods for managing dependencies. This can happen at runtime or at setup where the packages are pre-installed. The specifics for the different cluster managers are covered in Chapter 12.

With Kubernetes, for example, the default startup script checks for the presence of some key environment variables (EXTRA_APT_PACKAGES, EXTRA_CONDA_PACKAGES, and EXTRA_PIP_PACKAGES), which, in conjunction with customized worker specs, can be used to add dependencies at runtime. Some of them, like Coiled and Kubernetes, allow for adding dependencies when building an image for our workers. Others, like YARN, use preallocated conda/virtual environment packing.